In [1]:
state = {
    'market_price_XXXETH': 0.005, # starting market price: 1 ETH = 200 DAI
    'XXX_balance': int(100000000000000000000000),
    'ETH_balance': int(500000000000000000000) # initial liquidity pool: 500 ETH + 100.000 DAI
}
# state['ETH_balance'] = state['XXX_balance'] * state['market_price_XXXETH']
state['UNI_supply'] = state['ETH_balance']
state

{'market_price_XXXETH': 0.005,
 'XXX_balance': 100000000000000000000000,
 'ETH_balance': 500000000000000000000,
 'UNI_supply': 500000000000000000000}

In [2]:
params = {
    'fee_factor': [997, 996],
    'fee_base': [1000]
}
params

{'fee_factor': [997, 996], 'fee_base': [1000]}

In [3]:
# def addLiquidity(min_liquidity: uint256, max_tokens: uint256, deadline: timestamp) -> uint256:
#     assert deadline > block.timestamp and (max_tokens > 0 and msg.value > 0)
#     total_liquidity: uint256 = self.totalSupply
#     if total_liquidity > 0:
#         assert min_liquidity > 0
#         eth_reserve: uint256(wei) = self.balance - msg.value
#         token_reserve: uint256 = self.token.balanceOf(self)
#         token_amount: uint256 = msg.value * token_reserve / eth_reserve + 1
#         liquidity_minted: uint256 = msg.value * total_liquidity / eth_reserve
#         assert max_tokens >= token_amount and liquidity_minted >= min_liquidity
#         self.balances[msg.sender] += liquidity_minted
#         self.totalSupply = total_liquidity + liquidity_minted
#         assert self.token.transferFrom(msg.sender, self, token_amount)
#         log.AddLiquidity(msg.sender, msg.value, token_amount)
#         log.Transfer(ZERO_ADDRESS, msg.sender, liquidity_minted)
#         return liquidity_minted

def addLiquidity_UNI(params, step, history, current_state, input_):
    total_liquidity = current_state['UNI_supply']
    eth_reserve = current_state['ETH_balance']
    liquidity_minted = input_['eth_deposit'] * total_liquidity / eth_reserve
    return ('UNI_supply', total_liquidity + liquidity_minted)

def addLiquidity_ETH(params, step, history, current_state, input_):
    eth_reserve = current_state['ETH_balance']
    return ('ETH_balance', eth_reserve + input_['eth_deposit'])

def addLiquidity_XXX(params, step, history, current_state, input_):
    total_liquidity = current_state['UNI_supply']
    eth_reserve = current_state['ETH_balance']
    token_reserve = current_state['XXX_balance']
    token_amount = input_['eth_deposit'] * token_reserve / eth_reserve + 1
    return ('XXX_balance', token_reserve + token_amount)

In [4]:
# def removeLiquidity(amount: uint256, min_eth: uint256(wei), min_tokens: uint256, deadline: timestamp) -> (uint256(wei), uint256):
#     assert (amount > 0 and deadline > block.timestamp) and (min_eth > 0 and min_tokens > 0)
#     total_liquidity: uint256 = self.totalSupply
#     assert total_liquidity > 0
#     token_reserve: uint256 = self.token.balanceOf(self)
#     eth_amount: uint256(wei) = amount * self.balance / total_liquidity
#     token_amount: uint256 = amount * token_reserve / total_liquidity
#     assert eth_amount >= min_eth and token_amount >= min_tokens
#     self.balances[msg.sender] -= amount
#     self.totalSupply = total_liquidity - amount
#     send(msg.sender, eth_amount)
#     assert self.token.transfer(msg.sender, token_amount)
#     log.RemoveLiquidity(msg.sender, eth_amount, token_amount)
#     log.Transfer(msg.sender, ZERO_ADDRESS, amount)
#     return eth_amount, token_amount

def removeLiquidity_UNI(params, step, history, current_state, input_):
    total_liquidity = current_state['UNI_supply']
    amount = input_['UNI_burn']
    return ('UNI_supply', total_liquidity - amount)

def removeLiquidity_ETH(params, step, history, current_state, input_):
    total_liquidity = current_state['UNI_supply']
    eth_reserve = current_state['ETH_balance']
    amount = input_['UNI_burn']
    eth_amount = amount * eth_reserve / total_liquidity
    return ('ETH_balance', eth_reserve - eth_amount)

def removeLiquidity_XXX(params, step, history, current_state, input_):
    total_liquidity = current_state['UNI_supply']
    token_reserve = current_state['XXX_balance']
    amount = input_['UNI_burn']
    token_amount = amount * token_reserve / total_liquidity
    return ('XXX_balance', token_reserve - token_amount)

In [5]:
# @dev Pricing function for converting between ETH and Tokens.
# @param input_amount Amount of ETH or Tokens being sold.
# @param input_reserve Amount of ETH or Tokens (input type) in exchange reserves.
# @param output_reserve Amount of ETH or Tokens (output type) in exchange reserves.
# @return Amount of ETH or Tokens bought.
def getInputPrice(input_amount, input_reserve, output_reserve, params):
    fee_factor = params['fee_factor']
    fee_base = params['fee_base']
    input_amount_with_fee = input_amount * fee_factor
    numerator = input_amount_with_fee * output_reserve
    denominator = (input_reserve * fee_base) + input_amount_with_fee
    return numerator / denominator

# @dev Pricing function for converting between ETH and Tokens.
# @param output_amount Amount of ETH or Tokens being bought.
# @param input_reserve Amount of ETH or Tokens (input type) in exchange reserves.
# @param output_reserve Amount of ETH or Tokens (output type) in exchange reserves.
# @return Amount of ETH or Tokens sold.
def getOutputPrice(output_amount, input_reserve, output_reserve, params):
    fee_factor = params['fee_factor']
    fee_base = params['fee_base']
    numerator = input_reserve * output_amount * fee_base
    denominator = (output_reserve - output_amount) * fee_factor
    return numerator / denominator + 1

In [6]:
# @private
# def ethToTokenInput(eth_sold: uint256(wei), min_tokens: uint256, deadline: timestamp, buyer: address, recipient: address) -> uint256:
#     assert deadline >= block.timestamp and (eth_sold > 0 and min_tokens > 0)
#     token_reserve: uint256 = self.token.balanceOf(self)
#     tokens_bought: uint256 = self.getInputPrice(as_unitless_number(eth_sold), as_unitless_number(self.balance - eth_sold), token_reserve)
#     assert tokens_bought >= min_tokens
#     assert self.token.transfer(recipient, tokens_bought)
#     log.TokenPurchase(buyer, eth_sold, tokens_bought)
#     return tokens_bought

def ethToToken_ETH(params, step, history, current_state, input_):
    eth_sold = input_['eth_sold'] #amount of ETH being sold by the user
    eth_reserve = current_state['ETH_balance']
    return ('ETH_balance', eth_reserve + eth_sold)

def ethToToken_XXX(params, step, history, current_state, input_):
    eth_sold = input_['eth_sold'] #amount of ETH being sold by the user
    eth_reserve = current_state['ETH_balance']
    token_reserve = current_state['XXX_balance']
    tokens_bought = getInputPrice(eth_sold, eth_reserve, token_reserve, params)
    return ('XXX_balance', token_reserve - tokens_bought)

In [7]:
# @private
# def tokenToEthInput(tokens_sold: uint256, min_eth: uint256(wei), deadline: timestamp, buyer: address, recipient: address) -> uint256(wei):
#     assert deadline >= block.timestamp and (tokens_sold > 0 and min_eth > 0)
#     token_reserve: uint256 = self.token.balanceOf(self)
#     eth_bought: uint256 = self.getInputPrice(tokens_sold, token_reserve, as_unitless_number(self.balance))
#     wei_bought: uint256(wei) = as_wei_value(eth_bought, 'wei')
#     assert wei_bought >= min_eth
#     send(recipient, wei_bought)
#     assert self.token.transferFrom(buyer, self, tokens_sold)
#     log.EthPurchase(buyer, tokens_sold, wei_bought)
#     return wei_bought

def tokenToEth_ETH(params, step, history, current_state, input_):
    tokens_sold = input_['tokens_sold'] #amount of tokens being sold by the user
    eth_reserve = current_state['ETH_balance']
    token_reserve = current_state['XXX_balance']
    eth_bought = getInputPrice(tokens_sold, token_reserve, eth_reserve, params)
    return ('ETH_balance', eth_reserve - eth_bought)
    
def tokenToEth_XXX(params, step, history, current_state, input_):
    tokens_sold = input_['tokens_sold'] #amount of tokens being sold by the user
    token_reserve = current_state['XXX_balance']
    return ('XXX_balance', token_reserve + tokens_sold)

In [8]:
def marketPriceUpdate(params, step, history, current_state, input_):
    price = current_state['market_price_XXXETH']
    return ('market_price_XXXETH', price*0.999)

In [ ]:
PSUBs = [
    # substep 1 - market price
    {
        'policies': {
        },
        'variables': {
            'market_price_XXXETH': marketPriceUpdate
        }
    },
    # substep 2 - logical traders
    {
        'policies': {
            
        },
        'variables': {
            
        }
    },
    # substep 3 - liquidity providers
    {
        'policies': {
            
        },
        'variables': {
            
        }
    },
]
